In [ ]:
# In[1]:

# ### 导入库

import json
import requests
import pandas as pd 
import pyecharts.options as opts
from pyecharts.charts import *
from pyecharts.globals import ThemeType#设定主题
from pyecharts.commons.utils import JsCode
import chardet 
import jieba
import numpy as np

In [ ]:
# In[2]:

# ### 读取文件考研历年国家分数线

df1 = pd.read_csv(r'./考研历年国家分数线(1).csv')
df2 = pd.read_csv(r'./考研历年国家分数线(2).csv')
df3 = pd.read_csv(r'./考研历年国家分数线(3).csv')
df4 = pd.read_csv(r'./考研历年国家分数线(4).csv')
df5 = pd.read_csv(r'./考研历年国家分数线(5).csv')
df6 = pd.read_csv(r'./考研历年国家分数线(6).csv')
df_all= pd.concat([df1,df2,df3,df4,df5,df6])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100864 entries, 0 to 863
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   年份       100864 non-null  int64 
 1   学校名称_链接  100864 non-null  object
 2   学校名称     100863 non-null  object
 3   院系名称_链接  100864 non-null  object
 4   院系名称     100725 non-null  object
 5   专业代码     100761 non-null  object
 6   专业名称_链接  100864 non-null  object
 7   专业名称     100864 non-null  object
 8   总分       100864 non-null  object
 9   政治__管综   100864 non-null  object
 10  外语       100864 non-null  object
 11  业务课_一    100864 non-null  object
 12  业务课_二    100864 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.8+ MB


In [ ]:
# In[3]:

# ### 打印 df_all 数据框的形状信息，查看数据的行数和列数

print(df_all.shape)

(100864, 13)


In [ ]:
# In[4]:

# ### 检查并打印数据中的重复值和空值情况

print('重复值：' ,df_all.duplicated().sum())
print('空值: \n',df_all.isnull().sum())

重复值： 58
空值: 
 年份           0
学校名称_链接      0
学校名称         1
院系名称_链接      0
院系名称       139
专业代码       103
专业名称_链接      0
专业名称         0
总分           0
政治__管综       0
外语           0
业务课_一        0
业务课_二        0
dtype: int64
空值: 
 年份           0
学校名称_链接      0
学校名称         1
院系名称_链接      0
院系名称       139
专业代码       103
专业名称_链接      0
专业名称         0
总分           0
政治__管综       0
外语           0
业务课_一        0
业务课_二        0
dtype: int64


In [ ]:
# In[5]:

# ### 处理重复值和空值

df_all = df_all.drop_duplicates()
df_all = df_all.dropna(axis=0,how='any')

In [ ]:
# In[6]:

# ### 查看并打印数据的基本信息，包括列名、数据类型

df_all.info()
print(df_all.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 100563 entries, 0 to 863
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   年份       100563 non-null  int64 
 1   学校名称_链接  100563 non-null  object
 2   学校名称     100563 non-null  object
 3   院系名称_链接  100563 non-null  object
 4   院系名称     100563 non-null  object
 5   专业代码     100563 non-null  object
 6   专业名称_链接  100563 non-null  object
 7   专业名称     100563 non-null  object
 8   总分       100563 non-null  object
 9   政治__管综   100563 non-null  object
 10  外语       100563 non-null  object
 11  业务课_一    100563 non-null  object
 12  业务课_二    100563 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.7+ MB
(100563, 13)


In [ ]:
# In[7]:

# ### 检查打印数据中的重复值和空值情况

print('重复值：' ,df_all.duplicated().sum())
print('空值: \n',df_all.isnull().sum())

重复值： 0
空值: 
 年份         0
学校名称_链接    0
学校名称       0
院系名称_链接    0
院系名称       0
专业代码       0
专业名称_链接    0
专业名称       0
总分         0
政治__管综     0
外语         0
业务课_一      0
业务课_二      0
dtype: int64


In [ ]:
# In[8]:

# ### 删除不需要的列并查看前两行的数据

df_all = df_all.drop(labels=['学校名称_链接','院系名称_链接','专业名称_链接'],axis=1)
df_all.head(2)

,年份,学校名称,院系名称,专业代码,专业名称,总分,政治__管综,外语,业务课_一,业务课_二
0,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-
1,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-


In [ ]:
# In[9]:

# ### 替换特殊字符并查看前两行的数据

df_all['专业名称'] = df_all['专业名称'].str.replace('\(专业学位\)','')
df_all['专业名称'] = df_all['专业名称'].str.replace('★','')
df_all.head(2)

<>:4: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\('
<ipython-input-9-c7d30dfb0a3d>:4: SyntaxWarning: invalid escape sequence '\('
  df_all['专业名称'] = df_all['专业名称'].str.replace('\(专业学位\)','')


,年份,学校名称,院系名称,专业代码,专业名称,总分,政治__管综,外语,业务课_一,业务课_二
0,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-
1,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-


In [ ]:
# In[10]:

# ### 单独筛选出2020年考研信息并查看数据详情

data_2020 = df_all[df_all['年份'] == 2020]
data_2020.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65808 entries, 0 to 5995
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   年份      65808 non-null  int64 
 1   学校名称    65808 non-null  object
 2   院系名称    65808 non-null  object
 3   专业代码    65808 non-null  object
 4   专业名称    65808 non-null  object
 5   总分      65808 non-null  object
 6   政治__管综  65808 non-null  object
 7   外语      65808 non-null  object
 8   业务课_一   65808 non-null  object
 9   业务课_二   65808 non-null  object
dtypes: int64(1), object(9)
memory usage: 5.5+ MB


In [ ]:
# In[11]:

# ### 统计专业

data_2020['专业名称'].value_counts()[:100]

专业名称
计算机科学与技术      877
管理科学与工程       844
数学            808
机械工程          687
控制科学与工程       680
             ... 
计算机应用技术       178
(专业学位)应用心理    178
网络空间安全        177
中西医结合临床       175
国际贸易学         173
Name: count, Length: 100, dtype: int64

In [ ]:
# In[12]:

# ### 分组归纳学校对应的专业数（专业可能是重复值）

data_2020.groupby('学校名称')['专业名称'].count().sort_values(ascending = False)[:100]

学校名称
北京大学      3093
复旦大学      1248
清华大学      1217
武汉大学       858
中国人民大学     758
          ... 
河南科技大学     211
华北理工大学     208
上海师范大学     207
吉林农业大学     205
沈阳农业大学     203
Name: 专业名称, Length: 100, dtype: int64

In [ ]:
# In[13]:

# ### 转化考研专业总分特殊值

def tranform_num(x):
    if '-' in x:
        return 0
    else:
        return x
    
data_2020['总分'] = data_2020['总分'].apply(lambda x:tranform_num(x) )
data_2020['总分'] = data_2020['总分'].astype('int')

<ipython-input-13-a75c7f2addc5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020['总分'] = data_2020['总分'].apply(lambda x:tranform_num(x) )
<ipython-input-13-a75c7f2addc5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020['总分'] = data_2020['总分'].astype('int')


In [ ]:
# In[14]:

# ### 分组归纳各专业的最高分，最低分，平均分

data_1 = data_2020.groupby('专业名称')['总分'].agg([np.mean, np.max,np.min])
data_1['mean'] = data_1['mean'].astype('int')
data_1 = data_1.sort_values(by=['mean'],ascending=False)[:50]
data_1

<ipython-input-14-4752e01102f7>:4: FutureWarning: The provided callable <function mean at 0x0000024AFB744220> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data_1 = data_2020.groupby('专业名称')['总分'].agg([np.mean, np.max,np.min])
<ipython-input-14-4752e01102f7>:4: FutureWarning: The provided callable <function max at 0x0000024AFB72B7E0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data_1 = data_2020.groupby('专业名称')['总分'].agg([np.mean, np.max,np.min])
<ipython-input-14-4752e01102f7>:4: FutureWarning: The provided callable <function min at 0x0000024AFB72B920> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  data_1 = dat

,mean,max,min
专业名称,,,
公共关系学,409,409,409
国别与区域研究,402,402,402
美术理论研究,397,397,397
信息艺术设计,390,390,390
国民经济动员学,388,400,376
高级秘书与行政助理学,386,386,386
公共组织与人力资源,385,385,385
中国政治,383,383,383
符号学,382,382,382


In [ ]:
# In[15]:

# ### 绘制各专业分数的柱状图

bar = Bar(init_opts=opts.InitOpts(theme='light',
                                    width='1000px',
                                    height='1200px')
                                    )

bar.add_xaxis(data_1.index.tolist())
bar.add_yaxis('最高分', 
               data_1['max'].tolist(),
               z_level=1,
               stack='1',
               category_gap='50%',
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(position='insideRight', formatter='{c} 分'),
               itemstyle_opts={"normal": {
                        'shadowBlur': 10,
                        'shadowColor': 'rgba(0, 0, 0, 0.1)',
                        'shadowOffsetX': 10,
                        'shadowOffsetY': 10,
                        'color':'#ec9bad',
                        'borderColor': 'rgb(220,220,220)',
                        'borderWidth':2}
                },
               )
bar.add_yaxis('最低分', 
               data_1['min'].tolist(),
               z_level=1,
               stack='1',
               category_gap='50%',
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(position='insideLeft', formatter='{c} 分'),
               itemstyle_opts={"normal": {
                        'shadowBlur': 10,
                        'shadowColor': 'rgba(0, 0, 0, 0.1)',
                        'shadowOffsetX': 10,
                        'shadowOffsetY': 10,
                        'color':'#87CEFA',
                        'borderColor': 'rgb(220,220,220)',
                        'borderWidth':2}
                },
               )


bar.set_global_opts(title_opts=opts.TitleOpts(title="各专业的最高分和最低分",
                                              pos_left="center",
                                              pos_top='0%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=20,
                                                                                      color='#00BFFF')),
                        legend_opts=opts.LegendOpts(is_show=True, pos_top='3%'),
                        datazoom_opts=opts.DataZoomOpts(type_='inside',
                                                    range_start=50,   # 设置起止位置，50%-100%
                                                    range_end=100,
                                                    orient='vertical'),
                        xaxis_opts=opts.AxisOpts(is_show=False, max_=818),
                        yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(is_show=False),
                                             axistick_opts=opts.AxisTickOpts(is_show=False),
                                             axislabel_opts=opts.LabelOpts(color='#528B8B', font_size=10, font_weight='bold')),
                    )
bar.reversal_axis()
bar.render_notebook()

In [ ]:
# In[16]:

# ### 统计 2020 年数据中各专业名称的出现次数，并提取前 50 个出现次数最多的专业名称

data_2 = data_2020['专业名称'].value_counts()[:50]

In [ ]:
# In[17]:

# ### 绘制2020年考研专业Top50

data_x =data_2.index.tolist()
data_y = data_2.values.tolist()

bar = Bar(init_opts=opts.InitOpts(theme='light',
                                  width='1000px',
                                  height='900px'))
bar.add_xaxis(data_x)
bar.add_yaxis('考研专业', [int(i) for i in data_y])
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=12,
                                              font_weight='bold',
                                              formatter='{b}:{c} 个'))
bar.set_global_opts(title_opts=opts.TitleOpts(title="2020年考研专业Top50", pos_top='2%', pos_left='center', 
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=20,
                                                                                      color='#00BFFF')),
                    legend_opts=opts.LegendOpts(is_show=False),
                    xaxis_opts=opts.AxisOpts(is_show=False, is_scale=True),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    datazoom_opts=opts.DataZoomOpts(type_='inside',
                                                    range_start=50,   # 设置起止位置，50%-100%
                                                    range_end=100,
                                                    orient='vertical'),
                    
                    visualmap_opts=opts.VisualMapOpts(is_show=False, 
                                          max_=1058,
                                          min_=1,
                                          is_piecewise=False,
                                          dimension=0,
                                          range_color=['rgba(236,155,173,1)', 'rgba(237,157,178,0.4)'])
                                      )
bar.reversal_axis()
bar.render_notebook()

In [ ]:
# In[18]:

# ### 读取2021年考研调剂大学信息并查看前几行数据的信息

df_info = pd.read_excel(r'./大学信息2021new.xlsx')
df_info.head()

,school,school_type,school_attr,province
0,北方民族大学,民族类,中央部委高校\n\n\n丝绸之路大学联盟成员,宁夏
1,上海船舶运输科学研究所,NaN,NaN,上海
2,北华大学,综合,（2012年）\n（2014年\n全国高校实践育人创新创业基地（2015年）\n（2017年...,吉林
3,佛山科学技术学院,NaN,NaN,广东
4,武汉轻工大学,理工,国家粮食和物资储备局与湖北省人民政府共建高校（2018）\n湖北省国内一流学科建设高校（20...,湖北


In [ ]:
# In[19]:

# ### openpyxl库的安装

get_ipython().system('pip install openpyxl -i https://pypi.tuna.tsinghua.edu.cn/simple  --trusted-host pypi.tuna.tsinghua.edu.cn')

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\19398\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# In[20]:

# ### 查看数据的基本信息

df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   school       747 non-null    object
 1   school_type  528 non-null    object
 2   school_attr  531 non-null    object
 3   province     747 non-null    object
dtypes: object(4)
memory usage: 23.5+ KB


In [ ]:
# In[21]:

# ### 转换学校属性类别

#转换学校属性
def transform_attr(x):
    if '211' in x and '985' not in x:
        return 211 
    elif '985' in x:
        return '985'
    else:
        return '双非'

#转换学校类型   
def transform_type(x):
    if '理工类' in x or '理工类院校' in x or '理工科' in x or '理工、教学研究型大学' in x or '理工类\n[4]' in x or '理工\n[6]' in x:
        return '理工'
    elif '综合类' in x or '综合性大学\n[3]' in x or '综合类（应用型大学）' in x or '综合、研究教学型大学' in x or '综合类大学' in x or '综合师范类' in x:
        return '综合'
    elif '师范类院校' in x or '师范类' in x or '师范类（综合类）' in x or '师范（综合）' in x or '地方师范院校' in x:
        return '师范'
    elif '农林类' in x or '农业类' in x: 
        return '农林'
    elif '医药类' in x:
        return '医药'
    elif '民族类' in x:
        return '民族'
    elif '未知' in x or '国有企业' in x or '科技型企业' in x or '公立大学' in x:
        return '其他'
    elif '重点' in x or '省' in x or '2' in x or '' in x:
        return '其他'
    else:
        return x 
    
# 转换数据
df_info['school_level'] = df_info.school_attr.astype(str).apply(lambda x:transform_attr(x))
df_info['school_types'] = df_info.school_type.astype(str).apply(lambda x: transform_type(x))

# 筛选字段
df_info= df_info[['school','province','school_level','school_types']]

# 处理省份数据
df_info.loc[(df_info.school=='北京工商大学')&(df_info.province=='未知'), 'province']= '北京' 
df_info.loc[(df_info.school=='哈尔滨工程大学')&(df_info.province=='未知'), 'province']= '哈尔滨' 
df_info.loc[(df_info.school=='江苏大学')&(df_info.province=='未知'), 'province']= '江苏' 
df_info.loc[(df_info.school=='青岛大学')&(df_info.province=='未知'), 'province']= '山东' 
df_info.loc[(df_info.school=='北京石油化工学院')&(df_info.province=='未知'), 'province']= '北京' 
df_info.loc[(df_info.school=='齐鲁工业大学')&(df_info.province=='未知'), 'province']= '山东'
df_info.loc[(df_info.school=='江苏科技大学')&(df_info.province=='未知'), 'province']= '江苏'
df_info.loc[(df_info.school=='浙江农林大学')&(df_info.province=='未知'), 'province']= '浙江'
df_info.loc[(df_info.school=='燕山大学')&(df_info.province=='未知'), 'province']= '河北'
df_info.loc[(df_info.school=='福州大学')&(df_info.province=='未知'), 'province']= '福建'
df_info.loc[(df_info.school=='内蒙古科技大学')&(df_info.province=='未知'), 'province']= '内蒙古'

In [ ]:
# In[22]:

# ### 查看数据的前几条信息

df_info.head()

,school,province,school_level,school_types
0,北方民族大学,宁夏,双非,民族
1,上海船舶运输科学研究所,上海,双非,其他
2,北华大学,吉林,双非,其他
3,佛山科学技术学院,广东,双非,其他
4,武汉轻工大学,湖北,双非,其他


In [ ]:
# In[23]:

# ### 删除重复值并查看数据的基本信息

df_info = df_info.drop_duplicates()
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327 entries, 0 to 745
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   school        327 non-null    object
 1   province      327 non-null    object
 2   school_level  327 non-null    object
 3   school_types  327 non-null    object
dtypes: object(4)
memory usage: 12.8+ KB


In [ ]:
# In[24]:

# ### 确认数据的行数和列数

df_info.shape

(327, 4)

In [ ]:
# In[25]:

# ### 读取考研调剂数据 Excel 文件并查看行数和列数

df = pd.read_excel(r'./考研调剂数据-3.08.xlsx')
df.shape

(747, 5)

In [ ]:
# In[26]:

# ### 筛选出 2021 年的调剂信息并查看行数和列数

df_2021 = df[df['time'].str.contains('2021')].copy()
df_2021.shape

(747, 5)

In [ ]:
# In[27]:

# ### 通过左连接将 df_2021 和 df_info 按照 'school' 列合并，以获取调剂信息和大学信息的结合数据

pd.merge(df_2021,df_info,how = 'left',on = 'school').shape

(774, 8)

In [ ]:
# In[28]:

# ### 通过左连接确保 df_2021 中的所有行都被保留，同时补充 df_info 中的相关信息，展示前五条信息

df_all = pd.merge(df_2021,df_info,how = 'left',on = 'school')
df_all.head(5)

,school,name,title,url,time,province,school_level,school_types
0,北方民族大学,电子科学与技术、电子信息,2021年北方民族大学“电子科学与技术”和“电子信息”..,/tiaoji/schooldetail/id/28535.shtml,2021-03-09,宁夏,双非,民族
1,上海船舶运输科学研究所,轮机工程、通信与信息系统..,上海船舶运输科学研究所2021年硕士研究生招生调剂信息,/tiaoji/schooldetail/id/29191.shtml,2021-03-08,上海,双非,其他
2,北华大学,基础数学、计算数学、概率..,北华大学数学与统计学院2021年硕士研究生招生预调剂信..,/tiaoji/schooldetail/id/29189.shtml,2021-03-08,吉林,双非,其他
3,佛山科学技术学院,机械工程、光学工程、材料..,佛山科学技术学院2021年硕士研究生预调剂公告(一),/tiaoji/schooldetail/id/29187.shtml,2021-03-08,广东,双非,其他
4,佛山科学技术学院,机械工程、光学工程、材料..,佛山科学技术学院2021年硕士研究生预调剂公告(一),/tiaoji/schooldetail/id/29187.shtml,2021-03-08,湖南,双非,其他


In [ ]:
# In[29]:

# ### 重新排列数据，保留包括学校名称、调剂信息名称、时间、省份、学校层次和学校类型

df_all = df_all[['school','name','time','province','school_level','school_types']]
df_all.head()

,school,name,time,province,school_level,school_types
0,北方民族大学,电子科学与技术、电子信息,2021-03-09,宁夏,双非,民族
1,上海船舶运输科学研究所,轮机工程、通信与信息系统..,2021-03-08,上海,双非,其他
2,北华大学,基础数学、计算数学、概率..,2021-03-08,吉林,双非,其他
3,佛山科学技术学院,机械工程、光学工程、材料..,2021-03-08,广东,双非,其他
4,佛山科学技术学院,机械工程、光学工程、材料..,2021-03-08,湖南,双非,其他


In [ ]:
# In[30]:

# ### 查看缺失数据

df_all.isnull().sum()

school           0
name            10
time             0
province         0
school_level     0
school_types     0
dtype: int64

In [ ]:
# In[31]:

# ### 发布时间对应的发布频次

pub_time = df_all.time.value_counts().sort_index()
pub_time

time
2021-01-04      3
2021-01-05      1
2021-01-12      1
2021-01-19      2
2021-01-26      2
2021-02-05      1
2021-02-07      3
2021-02-18      6
2021-02-22      1
2021-02-23      3
2021-02-24      1
2021-02-25     14
2021-02-26    110
2021-02-27     64
2021-02-28     29
2021-03-01     78
2021-03-02     99
2021-03-03    101
2021-03-04     57
2021-03-05    108
2021-03-06     32
2021-03-07     12
2021-03-08     45
2021-03-09      1
Name: count, dtype: int64

In [ ]:
# In[32]:

# ### 调剂信息发布时间走势图

line1 = Line(init_opts=opts.InitOpts(width='1000px',height='600px'))
line1.add_xaxis(pub_time.index.tolist())
line1.add_yaxis('发布热度',pub_time.values.tolist(),
               areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
               label_opts=opts.LabelOpts(is_show=True))
line1.set_global_opts(title_opts=opts.TitleOpts(title='调剂信息发布时间走势图'),
                     toolbox_opts=opts.ToolboxOpts(),
                      xaxis_opts=opts.AxisOpts(name='时间',
                                               type_='category',                                           
                                               axislabel_opts=opts.LabelOpts(rotate=45),
                                               ),
                     visualmap_opts=opts.VisualMapOpts())
line1.render_notebook()

In [ ]:
# In[33]:

# ### 计算学校层次（school_level）的百分比分布

level_perc = df_all.school_level.value_counts() / df_all.school_level.value_counts().sum()
display(level_perc)
level_perc = np.round(level_perc * 100 ,2)
level_perc

school_level
双非     0.981912
985    0.011628
211    0.006460
Name: count, dtype: float64

school_level
双非     98.19
985     1.16
211     0.65
Name: count, dtype: float64

In [ ]:
# In[34]:

# ### 绘制学校类别饼图

pie1 = Pie(init_opts=opts.InitOpts(theme='light',width='800px',height='600px'))
pie1.add("", 
         [*zip(level_perc.index, level_perc.values)], 
         radius=["40%","75%"]) 
pie1.set_global_opts(title_opts=opts.TitleOpts(title='学校层次分布',pos_left='center', pos_top='center',title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#00BFFF', font_size=30, font_weight='bold'),
                                               ), 
                     legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),
#                      toolbox_opts=opts.ToolboxOpts()
                    )   
pie1.set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%")) 
pie1.render_notebook()

In [ ]:
# In[1]:
# ### 导入库

import json
import requests
import pandas as pd 
import pyecharts.options as opts
from pyecharts.charts import *
from pyecharts.globals import ThemeType#设定主题
from pyecharts.commons.utils import JsCode
import chardet 
import jieba
import numpy as np

In [ ]:
# In[2]:

# ### 读取文件考研历年国家分数线

df1 = pd.read_csv(r'./考研历年国家分数线(1).csv')
df2 = pd.read_csv(r'./考研历年国家分数线(2).csv')
df3 = pd.read_csv(r'./考研历年国家分数线(3).csv')
df4 = pd.read_csv(r'./考研历年国家分数线(4).csv')
df5 = pd.read_csv(r'./考研历年国家分数线(5).csv')
df6 = pd.read_csv(r'./考研历年国家分数线(6).csv')
df_all= pd.concat([df1,df2,df3,df4,df5,df6])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100864 entries, 0 to 863
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   年份       100864 non-null  int64 
 1   学校名称_链接  100864 non-null  object
 2   学校名称     100863 non-null  object
 3   院系名称_链接  100864 non-null  object
 4   院系名称     100725 non-null  object
 5   专业代码     100761 non-null  object
 6   专业名称_链接  100864 non-null  object
 7   专业名称     100864 non-null  object
 8   总分       100864 non-null  object
 9   政治__管综   100864 non-null  object
 10  外语       100864 non-null  object
 11  业务课_一    100864 non-null  object
 12  业务课_二    100864 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.8+ MB


In [ ]:
# In[3]:

# ### 打印 df_all 数据框的形状信息，查看数据的行数和列数

print(df_all.shape)

(100864, 13)


In [ ]:
# In[4]:

# ### 检查并打印数据中的重复值和空值情况

print('重复值：' ,df_all.duplicated().sum())
print('空值: \n',df_all.isnull().sum())

重复值： 58
空值: 
 年份           0
学校名称_链接      0
学校名称         1
院系名称_链接      0
院系名称       139
专业代码       103
专业名称_链接      0
专业名称         0
总分           0
政治__管综       0
外语           0
业务课_一        0
业务课_二        0
dtype: int64


In [ ]:
# In[5]:

# ### 处理重复值和空值

df_all = df_all.drop_duplicates()
df_all = df_all.dropna(axis=0,how='any')

In [ ]:
# In[6]:

# ### 查看并打印数据的基本信息，包括列名、数据类型

df_all.info()
print(df_all.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 100563 entries, 0 to 863
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   年份       100563 non-null  int64 
 1   学校名称_链接  100563 non-null  object
 2   学校名称     100563 non-null  object
 3   院系名称_链接  100563 non-null  object
 4   院系名称     100563 non-null  object
 5   专业代码     100563 non-null  object
 6   专业名称_链接  100563 non-null  object
 7   专业名称     100563 non-null  object
 8   总分       100563 non-null  object
 9   政治__管综   100563 non-null  object
 10  外语       100563 non-null  object
 11  业务课_一    100563 non-null  object
 12  业务课_二    100563 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.7+ MB
(100563, 13)


In [ ]:
# In[7]:

# ### 检查打印数据中的重复值和空值情况

print('重复值：' ,df_all.duplicated().sum())
print('空值: \n',df_all.isnull().sum())

重复值： 0
空值: 
 年份         0
学校名称_链接    0
学校名称       0
院系名称_链接    0
院系名称       0
专业代码       0
专业名称_链接    0
专业名称       0
总分         0
政治__管综     0
外语         0
业务课_一      0
业务课_二      0
dtype: int64


In [ ]:
# In[8]:

# ### 删除不需要的列并查看前两行的数据

df_all = df_all.drop(labels=['学校名称_链接','院系名称_链接','专业名称_链接'],axis=1)
df_all.head(2)

,年份,学校名称,院系名称,专业代码,专业名称,总分,政治__管综,外语,业务课_一,业务课_二
0,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-
1,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-


In [ ]:
# In[9]:

# ### 替换特殊字符并查看前两行的数据

df_all['专业名称'] = df_all['专业名称'].str.replace('\(专业学位\)','')
df_all['专业名称'] = df_all['专业名称'].str.replace('★','')
df_all.head(2)

<>:4: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\('
<ipython-input-43-c7d30dfb0a3d>:4: SyntaxWarning: invalid escape sequence '\('
  df_all['专业名称'] = df_all['专业名称'].str.replace('\(专业学位\)','')


,年份,学校名称,院系名称,专业代码,专业名称,总分,政治__管综,外语,业务课_一,业务课_二
0,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-
1,2020,中国人民大学,公共管理学院,125200,(专业学位)公共管理,175,88,44,-,-


In [ ]:
# In[10]:

# ### 单独筛选出2020年考研信息并查看数据详情

data_2020 = df_all[df_all['年份'] == 2020]
data_2020.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65808 entries, 0 to 5995
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   年份      65808 non-null  int64 
 1   学校名称    65808 non-null  object
 2   院系名称    65808 non-null  object
 3   专业代码    65808 non-null  object
 4   专业名称    65808 non-null  object
 5   总分      65808 non-null  object
 6   政治__管综  65808 non-null  object
 7   外语      65808 non-null  object
 8   业务课_一   65808 non-null  object
 9   业务课_二   65808 non-null  object
dtypes: int64(1), object(9)
memory usage: 5.5+ MB


In [ ]:
# In[11]:

# ### 统计专业

data_2020['专业名称'].value_counts()[:100]

专业名称
计算机科学与技术      877
管理科学与工程       844
数学            808
机械工程          687
控制科学与工程       680
             ... 
计算机应用技术       178
(专业学位)应用心理    178
网络空间安全        177
中西医结合临床       175
国际贸易学         173
Name: count, Length: 100, dtype: int64

In [ ]:
# In[12]:

# ### 分组归纳学校对应的专业数（专业可能是重复值）

data_2020.groupby('学校名称')['专业名称'].count().sort_values(ascending = False)[:100]

学校名称
北京大学      3093
复旦大学      1248
清华大学      1217
武汉大学       858
中国人民大学     758
          ... 
河南科技大学     211
华北理工大学     208
上海师范大学     207
吉林农业大学     205
沈阳农业大学     203
Name: 专业名称, Length: 100, dtype: int64

In [ ]:
# In[13]:

# ### 转化考研专业总分特殊值

def tranform_num(x):
    if '-' in x:
        return 0
    else:
        return x
    
data_2020['总分'] = data_2020['总分'].apply(lambda x:tranform_num(x) )
data_2020['总分'] = data_2020['总分'].astype('int')

<ipython-input-47-a75c7f2addc5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020['总分'] = data_2020['总分'].apply(lambda x:tranform_num(x) )
<ipython-input-47-a75c7f2addc5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020['总分'] = data_2020['总分'].astype('int')


In [ ]:
# In[14]:

# ### 分组归纳各专业的最高分，最低分，平均分

data_1 = data_2020.groupby('专业名称')['总分'].agg([np.mean, np.max,np.min])
data_1['mean'] = data_1['mean'].astype('int')
data_1 = data_1.sort_values(by=['mean'],ascending=False)[:50]
data_1

<ipython-input-48-4752e01102f7>:4: FutureWarning: The provided callable <function mean at 0x0000024AFB744220> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data_1 = data_2020.groupby('专业名称')['总分'].agg([np.mean, np.max,np.min])
<ipython-input-48-4752e01102f7>:4: FutureWarning: The provided callable <function max at 0x0000024AFB72B7E0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data_1 = data_2020.groupby('专业名称')['总分'].agg([np.mean, np.max,np.min])
<ipython-input-48-4752e01102f7>:4: FutureWarning: The provided callable <function min at 0x0000024AFB72B920> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  data_1 = dat

,mean,max,min
专业名称,,,
公共关系学,409,409,409
国别与区域研究,402,402,402
美术理论研究,397,397,397
信息艺术设计,390,390,390
国民经济动员学,388,400,376
高级秘书与行政助理学,386,386,386
公共组织与人力资源,385,385,385
中国政治,383,383,383
符号学,382,382,382


In [ ]:
# In[15]:

# ### 绘制各专业分数的柱状图

bar = Bar(init_opts=opts.InitOpts(theme='light',
                                    width='1000px',
                                    height='1200px')
                                    )

bar.add_xaxis(data_1.index.tolist())
bar.add_yaxis('最高分', 
               data_1['max'].tolist(),
               z_level=1,
               stack='1',
               category_gap='50%',
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(position='insideRight', formatter='{c} 分'),
               itemstyle_opts={"normal": {
                        'shadowBlur': 10,
                        'shadowColor': 'rgba(0, 0, 0, 0.1)',
                        'shadowOffsetX': 10,
                        'shadowOffsetY': 10,
                        'color':'#ec9bad',
                        'borderColor': 'rgb(220,220,220)',
                        'borderWidth':2}
                },
               )
bar.add_yaxis('最低分', 
               data_1['min'].tolist(),
               z_level=1,
               stack='1',
               category_gap='50%',
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(position='insideLeft', formatter='{c} 分'),
               itemstyle_opts={"normal": {
                        'shadowBlur': 10,
                        'shadowColor': 'rgba(0, 0, 0, 0.1)',
                        'shadowOffsetX': 10,
                        'shadowOffsetY': 10,
                        'color':'#87CEFA',
                        'borderColor': 'rgb(220,220,220)',
                        'borderWidth':2}
                },
               )


bar.set_global_opts(title_opts=opts.TitleOpts(title="各专业的最高分和最低分",
                                              pos_left="center",
                                              pos_top='0%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=20,
                                                                                      color='#00BFFF')),
                        legend_opts=opts.LegendOpts(is_show=True, pos_top='3%'),
                        datazoom_opts=opts.DataZoomOpts(type_='inside',
                                                    range_start=50,   # 设置起止位置，50%-100%
                                                    range_end=100,
                                                    orient='vertical'),
                        xaxis_opts=opts.AxisOpts(is_show=False, max_=818),
                        yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(is_show=False),
                                             axistick_opts=opts.AxisTickOpts(is_show=False),
                                             axislabel_opts=opts.LabelOpts(color='#528B8B', font_size=10, font_weight='bold')),
                    )
bar.reversal_axis()
bar.render_notebook()

In [ ]:
# In[16]:

# ### 统计 2020 年数据中各专业名称的出现次数，并提取前 50 个出现次数最多的专业名称

data_2 = data_2020['专业名称'].value_counts()[:50]

In [ ]:
# In[17]:

# ### 绘制2020年考研专业Top50

data_x =data_2.index.tolist()
data_y = data_2.values.tolist()

bar = Bar(init_opts=opts.InitOpts(theme='light',
                                  width='1000px',
                                  height='900px'))
bar.add_xaxis(data_x)
bar.add_yaxis('考研专业', [int(i) for i in data_y])
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=12,
                                              font_weight='bold',
                                              formatter='{b}:{c} 个'))
bar.set_global_opts(title_opts=opts.TitleOpts(title="2020年考研专业Top50", pos_top='2%', pos_left='center', 
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=20,
                                                                                      color='#00BFFF')),
                    legend_opts=opts.LegendOpts(is_show=False),
                    xaxis_opts=opts.AxisOpts(is_show=False, is_scale=True),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    datazoom_opts=opts.DataZoomOpts(type_='inside',
                                                    range_start=50,   # 设置起止位置，50%-100%
                                                    range_end=100,
                                                    orient='vertical'),
                    
                    visualmap_opts=opts.VisualMapOpts(is_show=False, 
                                          max_=1058,
                                          min_=1,
                                          is_piecewise=False,
                                          dimension=0,
                                          range_color=['rgba(236,155,173,1)', 'rgba(237,157,178,0.4)'])
                                      )
bar.reversal_axis()
bar.render_notebook()

In [ ]:
# In[18]:

# ### 读取2021年考研调剂大学信息并查看前几行数据的信息

df_info = pd.read_excel(r'./大学信息2021new.xlsx')
df_info.head()

,school,school_type,school_attr,province
0,北方民族大学,民族类,中央部委高校\n\n\n丝绸之路大学联盟成员,宁夏
1,上海船舶运输科学研究所,NaN,NaN,上海
2,北华大学,综合,（2012年）\n（2014年\n全国高校实践育人创新创业基地（2015年）\n（2017年...,吉林
3,佛山科学技术学院,NaN,NaN,广东
4,武汉轻工大学,理工,国家粮食和物资储备局与湖北省人民政府共建高校（2018）\n湖北省国内一流学科建设高校（20...,湖北


In [ ]:
# In[19]:

# ### openpyxl库的安装

get_ipython().system('pip install openpyxl -i https://pypi.tuna.tsinghua.edu.cn/simple  --trusted-host pypi.tuna.tsinghua.edu.cn')

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\19398\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# In[20]:

# ### 查看数据的基本信息

df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   school       747 non-null    object
 1   school_type  528 non-null    object
 2   school_attr  531 non-null    object
 3   province     747 non-null    object
dtypes: object(4)
memory usage: 23.5+ KB


In [ ]:
# In[21]:

# ### 转换学校属性类别

#转换学校属性
def transform_attr(x):
    if '211' in x and '985' not in x:
        return 211 
    elif '985' in x:
        return '985'
    else:
        return '双非'

#转换学校类型   
def transform_type(x):
    if '理工类' in x or '理工类院校' in x or '理工科' in x or '理工、教学研究型大学' in x or '理工类\n[4]' in x or '理工\n[6]' in x:
        return '理工'
    elif '综合类' in x or '综合性大学\n[3]' in x or '综合类（应用型大学）' in x or '综合、研究教学型大学' in x or '综合类大学' in x or '综合师范类' in x:
        return '综合'
    elif '师范类院校' in x or '师范类' in x or '师范类（综合类）' in x or '师范（综合）' in x or '地方师范院校' in x:
        return '师范'
    elif '农林类' in x or '农业类' in x: 
        return '农林'
    elif '医药类' in x:
        return '医药'
    elif '民族类' in x:
        return '民族'
    elif '未知' in x or '国有企业' in x or '科技型企业' in x or '公立大学' in x:
        return '其他'
    elif '重点' in x or '省' in x or '2' in x or '' in x:
        return '其他'
    else:
        return x 
    
# 转换数据
df_info['school_level'] = df_info.school_attr.astype(str).apply(lambda x:transform_attr(x))
df_info['school_types'] = df_info.school_type.astype(str).apply(lambda x: transform_type(x))

# 筛选字段
df_info= df_info[['school','province','school_level','school_types']]

# 处理省份数据
df_info.loc[(df_info.school=='北京工商大学')&(df_info.province=='未知'), 'province']= '北京' 
df_info.loc[(df_info.school=='哈尔滨工程大学')&(df_info.province=='未知'), 'province']= '哈尔滨' 
df_info.loc[(df_info.school=='江苏大学')&(df_info.province=='未知'), 'province']= '江苏' 
df_info.loc[(df_info.school=='青岛大学')&(df_info.province=='未知'), 'province']= '山东' 
df_info.loc[(df_info.school=='北京石油化工学院')&(df_info.province=='未知'), 'province']= '北京' 
df_info.loc[(df_info.school=='齐鲁工业大学')&(df_info.province=='未知'), 'province']= '山东'
df_info.loc[(df_info.school=='江苏科技大学')&(df_info.province=='未知'), 'province']= '江苏'
df_info.loc[(df_info.school=='浙江农林大学')&(df_info.province=='未知'), 'province']= '浙江'
df_info.loc[(df_info.school=='燕山大学')&(df_info.province=='未知'), 'province']= '河北'
df_info.loc[(df_info.school=='福州大学')&(df_info.province=='未知'), 'province']= '福建'
df_info.loc[(df_info.school=='内蒙古科技大学')&(df_info.province=='未知'), 'province']= '内蒙古'

In [ ]:
# In[22]:

# ### 查看数据的前几条信息

df_info.head()

,school,province,school_level,school_types
0,北方民族大学,宁夏,双非,民族
1,上海船舶运输科学研究所,上海,双非,其他
2,北华大学,吉林,双非,其他
3,佛山科学技术学院,广东,双非,其他
4,武汉轻工大学,湖北,双非,其他


In [ ]:
# In[23]:

# ### 删除重复值并查看数据的基本信息

df_info = df_info.drop_duplicates()
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327 entries, 0 to 745
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   school        327 non-null    object
 1   province      327 non-null    object
 2   school_level  327 non-null    object
 3   school_types  327 non-null    object
dtypes: object(4)
memory usage: 12.8+ KB


In [ ]:
# In[24]:

# ### 确认数据的行数和列数

df_info.shape

(327, 4)

In [ ]:
# In[25]:

# ### 读取考研调剂数据 Excel 文件并查看行数和列数

df = pd.read_excel(r'./考研调剂数据-3.08.xlsx')
df.shape

(747, 5)

In [ ]:
# In[26]:

# ### 筛选出 2021 年的调剂信息并查看行数和列数

df_2021 = df[df['time'].str.contains('2021')].copy()
df_2021.shape

(747, 5)

In [ ]:
# In[27]:

# ### 通过左连接将 df_2021 和 df_info 按照 'school' 列合并，以获取调剂信息和大学信息的结合数据

pd.merge(df_2021,df_info,how = 'left',on = 'school').shape

(774, 8)

In [ ]:
# In[28]:

# ### 通过左连接确保 df_2021 中的所有行都被保留，同时补充 df_info 中的相关信息，展示前五条信息

df_all = pd.merge(df_2021,df_info,how = 'left',on = 'school')
df_all.head(5)

,school,name,title,url,time,province,school_level,school_types
0,北方民族大学,电子科学与技术、电子信息,2021年北方民族大学“电子科学与技术”和“电子信息”..,/tiaoji/schooldetail/id/28535.shtml,2021-03-09,宁夏,双非,民族
1,上海船舶运输科学研究所,轮机工程、通信与信息系统..,上海船舶运输科学研究所2021年硕士研究生招生调剂信息,/tiaoji/schooldetail/id/29191.shtml,2021-03-08,上海,双非,其他
2,北华大学,基础数学、计算数学、概率..,北华大学数学与统计学院2021年硕士研究生招生预调剂信..,/tiaoji/schooldetail/id/29189.shtml,2021-03-08,吉林,双非,其他
3,佛山科学技术学院,机械工程、光学工程、材料..,佛山科学技术学院2021年硕士研究生预调剂公告(一),/tiaoji/schooldetail/id/29187.shtml,2021-03-08,广东,双非,其他
4,佛山科学技术学院,机械工程、光学工程、材料..,佛山科学技术学院2021年硕士研究生预调剂公告(一),/tiaoji/schooldetail/id/29187.shtml,2021-03-08,湖南,双非,其他


In [ ]:
# In[29]:

# ### 重新排列数据，保留包括学校名称、调剂信息名称、时间、省份、学校层次和学校类型

df_all = df_all[['school','name','time','province','school_level','school_types']]
df_all.head()

,school,name,time,province,school_level,school_types
0,北方民族大学,电子科学与技术、电子信息,2021-03-09,宁夏,双非,民族
1,上海船舶运输科学研究所,轮机工程、通信与信息系统..,2021-03-08,上海,双非,其他
2,北华大学,基础数学、计算数学、概率..,2021-03-08,吉林,双非,其他
3,佛山科学技术学院,机械工程、光学工程、材料..,2021-03-08,广东,双非,其他
4,佛山科学技术学院,机械工程、光学工程、材料..,2021-03-08,湖南,双非,其他


In [ ]:
# In[30]:

# ### 查看缺失数据

df_all.isnull().sum()

school           0
name            10
time             0
province         0
school_level     0
school_types     0
dtype: int64

In [ ]:
# In[31]:

# ### 发布时间对应的发布频次

pub_time = df_all.time.value_counts().sort_index()
pub_time

time
2021-01-04      3
2021-01-05      1
2021-01-12      1
2021-01-19      2
2021-01-26      2
2021-02-05      1
2021-02-07      3
2021-02-18      6
2021-02-22      1
2021-02-23      3
2021-02-24      1
2021-02-25     14
2021-02-26    110
2021-02-27     64
2021-02-28     29
2021-03-01     78
2021-03-02     99
2021-03-03    101
2021-03-04     57
2021-03-05    108
2021-03-06     32
2021-03-07     12
2021-03-08     45
2021-03-09      1
Name: count, dtype: int64

In [ ]:
# In[32]:

# ### 调剂信息发布时间走势图

line1 = Line(init_opts=opts.InitOpts(width='1000px',height='600px'))
line1.add_xaxis(pub_time.index.tolist())
line1.add_yaxis('发布热度',pub_time.values.tolist(),
               areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
               label_opts=opts.LabelOpts(is_show=True))
line1.set_global_opts(title_opts=opts.TitleOpts(title='调剂信息发布时间走势图'),
                     toolbox_opts=opts.ToolboxOpts(),
                      xaxis_opts=opts.AxisOpts(name='时间',
                                               type_='category',                                           
                                               axislabel_opts=opts.LabelOpts(rotate=45),
                                               ),
                     visualmap_opts=opts.VisualMapOpts())
line1.render_notebook()

In [ ]:
# In[33]:

# ### 计算学校层次（school_level）的百分比分布

level_perc = df_all.school_level.value_counts() / df_all.school_level.value_counts().sum()
display(level_perc)
level_perc = np.round(level_perc * 100 ,2)
level_perc

school_level
双非     0.981912
985    0.011628
211    0.006460
Name: count, dtype: float64

school_level
双非     98.19
985     1.16
211     0.65
Name: count, dtype: float64

In [ ]:
# In[34]:

# ### 绘制学校类别饼图

pie1 = Pie(init_opts=opts.InitOpts(theme='light',width='800px',height='600px'))
pie1.add("", 
         [*zip(level_perc.index, level_perc.values)], 
         radius=["40%","75%"]) 
pie1.set_global_opts(title_opts=opts.TitleOpts(title='学校层次分布',pos_left='center', pos_top='center',title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#00BFFF', font_size=30, font_weight='bold'),
                                               ), 
                     legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),
#                      toolbox_opts=opts.ToolboxOpts()
                    )   
pie1.set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%")) 
pie1.render_notebook()